In [2]:
import numpy as np
import math
from Crypto.Util.number import getPrime, inverse, bytes_to_long, long_to_bytes, GCD
import random

# Prerequisites: 

1. RSA 

2. RSA Prerequisites

In [1]:
def create_key_rsa(p, q, e):
    assert GCD(e, (p-1)*(q-1)) == 1
    N = p*q
    return N, e
def encrypt_rsa(m, e, N):
    assert m < N
    c = pow(m, e, N)
    return c
def decrypt_rsa(c, p, q, e, N):
    d = inverse(e, (p-1)*(q-1))
    assert (d * e) % ((p-1)*(q-1)) == 1
    m = pow(c, d, N)
    return m

# $N$ is prime

If N is prime => $\varphi(N) = p-1$ => $de \equiv 1 \ mod \ N-1$

In [14]:
m = bytes_to_long(b'secret')
N = getPrime(1024)
e=65537

In [15]:
c = encrypt_rsa(m, e, N)
print(c)

12215353351590448535167310686476691358028024243197695861505238294015253293790161243769504462981163273048546128219537843351214237607897620764115410129869160009016149331981440482546004047134503134241242663603267819450699191965441684987593462148990407097818087217020032575908937458390338601878753051561686082297


In [18]:
phi = N-1
d = inverse(e, phi)
m_decr = pow(c, d, N)

In [19]:
long_to_bytes(m_decr)

b'secret'

# $N = p^2$

https://en.wikipedia.org/wiki/Euler%27s_totient_function#Value_for_a_prime_power_argument


In [20]:
def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

In [28]:
m = bytes_to_long(b'secret')
p = getPrime(512)
N = pow(p, 2)
e=65537

In [29]:
c = encrypt_rsa(m, e, N)
print(c)

10117983930442477388064413438183081758200087804351076023439762217159905576345235118448168509127881386741736288688514884350880843473649141957312207953815824734709106805693662040473947970815145974472500061315144875786521537960093777169817876825249999418301357189926495836503278394288264990406521066249828708998


In [30]:
p_decr = isqrt(N)
phi = p*(p-1)
d = inverse(e, phi)
m_decr = pow(c, d, N)
print(long_to_bytes(m_decr))

b'secret'


# e = 1

Then $c \equiv m^e \equiv m^1 \equiv m \ mod \ N$

# Using small N

You can factor N => factordb library


In [57]:
from factordb.factordb import FactorDB

In [64]:
p = getPrime(100)
q = getPrime(100)
N = p*q
print('p= ', p)
print('q= ', q)

p=  762519624440665876556479833467
q=  1106090959122647201209128911207


In [76]:
N

843416062747416855803616155832785095979008569880606389964669

In [74]:
f = FactorDB(N)
f.connect()

<Response [200]>

In [75]:
print(f.get_factor_list())

[762519624440665876556479833467, 1106090959122647201209128911207]


# Small e, very big N

In this scenario we might not reach the modulus => just take the e-th root

In [93]:
def iroot(k, n):
    u, s = n, n+1
    while u < s:
        s = u
        t = (k-1) * s + n // pow(s, k-1)
        u = t // k
    return s

In [97]:
e = 3
N = 17258212916191948536348548470938004244269544560039009244721959293554822498047075403658429865201816363311805874117705688359853941515579440852166618074161313773416434156467811969628473425365608002907061241714688204565170146117869742910273064909154666642642308154422770994836108669814632309362483307560217924183202838588431342622551598499747369771295105890359290073146330677383341121242366368309126850094371525078749496850520075015636716490087482193603562501577348571256210991732071282478547626856068209192987351212490642903450263288650415552403935705444809043563866466823492258216747445926536608548665086042098252335883
m = bytes_to_long(b'secret')
c = encrypt_rsa(m, e, N)

In [98]:
m_decr =iroot(3, c)
print(long_to_bytes(m_decr))

b'secret'


# Multiple N's have a common prime 

- Here we have $N_1, N_2 ...$ moduli with $C_1, C_2, ... $ ciphertexts encrypted with $e_1, e_2, ...$ corresponding public exponents
- If we find $\gcd(N_i, N_j) != 1$ for some $i!=j$ then we found $p = \gcd(N_i, N_j) => q_i = \frac {N_i}  p; \  q_j = \frac {N_j} p => (d_i, d_j) => (m_i, m_j)$

In [5]:
p = getPrime(512)
q1 = getPrime(512)
q2 = getPrime(512)
N1 = p*q1
N2 = p*q2
e1, e2 = 17,65537
m1, m2 = bytes_to_long(b'secret'), bytes_to_long(b'verysecret')
c1, c2 = pow(m1, e1, N1), pow(m2, e2, N2)

In [9]:
p_decr = GCD(N1, N2)
q1_decr = N1 // p_decr
q2_decr = N2 // p_decr
d1 = inverse(e1, (p-1)*(q1_decr-1))
d2 = inverse(e2, (p-1)*(q2_decr-1))
m1_decr = pow(c1, d1, N1)
m2_decr = pow(c2, d2, N2)

In [10]:
long_to_bytes(m1_decr), long_to_bytes(m2_decr)

(b'secret', b'verysecret')